In [16]:
from AC import preproc
from AC import get_news
import inspect
import nltk
nltk.download("stopwords")

%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\astefani\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
economia = get_news("../articoli_economia/")
cultura = get_news("../articoli_cultura/")
tech = get_news("../articoli_tech/")
politica = get_news("../articoli_politica/")
sport = get_news("../articoli_sport/")
cronaca = get_news("../articoli_cronaca/")

for articolo in economia:
    articolo['categoria'] = "Economia"
for articolo in cultura:
    articolo['categoria'] = "Cultura"
for articolo in tech:
    articolo['categoria'] = "Tech"
for articolo in politica:
    articolo['categoria'] = "Politica"
for articolo in sport:
    articolo['categoria'] = "Sport"
for articolo in cronaca:
    articolo['categoria'] = "Cronaca"
dati_preprocessati =  preproc(tech + politica + cultura + economia + sport + cronaca)

100%|██████████| 2400/2400 [00:16<00:00, 178.38it/s]


In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import ParameterGrid

import numpy as np
import pandas as pd
import math

import tqdm
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn import linear_model,metrics

In [19]:
seed = 0
train_documents,test_documents = train_test_split(dati_preprocessati,random_state=seed,train_size=0.5)
test_documents, val_documents = train_test_split(test_documents,random_state=seed,train_size=0.5)

C:\Python\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [20]:
docs_texts = [' '.join([word for word in x['testo']] + x['tags'] + x['sottotitolo'] + x['titolo_articolo']) for x in dati_preprocessati]
train_texts = [' '.join([word for word in x['testo']] + x['sottotitolo'] + x['titolo_articolo']) for x in train_documents]
test_texts = [' '.join([word for word in x['testo']] + x['sottotitolo'] + x['titolo_articolo']) for x in test_documents]
val_texts = [' '.join([word for word in x['testo']] + x['sottotitolo'] + x['titolo_articolo']) for x in val_documents]

docs_cats = [x["categoria"] for x in dati_preprocessati]
train_cats = [x["categoria"] for x in train_documents]
test_cats = [x["categoria"] for x in test_documents]
val_cats = [x["categoria"] for x in val_documents]

# Tempi con LDA-12

In [21]:
params_tdm = {
    'ngram_range': (1, 3),
    'min_df': 9,
    'max_df': 0.5
}

params_lda = {
    'n_components':12,
    'learning_decay':0.7    
}

def foo(params_lda, params_tdm):
    tf = CountVectorizer(max_features=1000000, lowercase=True, **params_tdm).fit(train_texts)
    tf_train = tf.transform(train_texts)
    ldax = LatentDirichletAllocation(max_iter=50, learning_method='online', random_state=0, **params_lda).fit(tf_train.toarray())
    ldax_train = ldax.transform(tf_train.toarray())
    tf_test = tf.transform(test_texts)
    ldax_test = ldax.transform(tf_test.toarray())

import timeit

REPEAT = 3
NUMBER = 1
tempi = timeit.repeat("foo(params_lda, params_tdm)",setup="from __main__ import foo,params_lda, params_tdm",repeat=REPEAT, number=NUMBER)
print("{:.2f} s ± {:.0f} ms per loop (mean ± std. dev. of {} runs, {} loop each)".format(
    np.mean(tempi), 1000 * np.std(tempi), REPEAT, NUMBER))

17.22 s ± 150 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [22]:
tf = CountVectorizer(max_features=1000000, lowercase=True, **params_tdm).fit(train_texts)
tf_train = tf.transform(train_texts)
ldax = LatentDirichletAllocation(max_iter=50, learning_method='online', random_state=0, **params_lda).fit(tf_train.toarray())
ldax_train = ldax.transform(tf_train.toarray())
tf_test = tf.transform(test_texts)
ldax_test = ldax.transform(tf_test.toarray())

params = {
    'max_depth':17,
    'min_samples_leaf':1 
}

def foo(params):
    ldac = DecisionTreeClassifier(random_state=0, **params).fit(ldax_train, train_cats)
    y_pred = ldac.predict(ldax_test)

REPEAT = 3
NUMBER = 1
tempi = timeit.repeat("foo(params)",setup="from __main__ import foo, params",repeat=REPEAT, number=NUMBER)
print("{:.2f} s ± {:.0f} ms per loop (mean ± std. dev. of {} runs, {} loop each)".format(
    np.mean(tempi), 1000 * np.std(tempi), REPEAT, NUMBER))

0.01 s ± 0 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


# Tempi con LDA-48

In [23]:
params_tdm = {
    'ngram_range': (1, 3),
    'min_df': 9,
    'max_df': 0.5
}

params_lda = {
    'n_components':48,
    'learning_decay':0.7    
}

def foo(params_lda, params_tdm):
    tf = CountVectorizer(max_features=1000000, lowercase=True, **params_tdm).fit(train_texts)
    tf_train = tf.transform(train_texts)
    ldax = LatentDirichletAllocation(max_iter=50, learning_method='online', random_state=0, **params_lda).fit(tf_train.toarray())
    ldax_train = ldax.transform(tf_train.toarray())
    tf_test = tf.transform(test_texts)
    ldax_test = ldax.transform(tf_test.toarray())

import timeit

REPEAT = 3
NUMBER = 1
tempi = timeit.repeat("foo(params_lda, params_tdm)",setup="from __main__ import foo,params_lda, params_tdm",repeat=REPEAT, number=NUMBER)
print("{:.2f} s ± {:.0f} ms per loop (mean ± std. dev. of {} runs, {} loop each)".format(
    np.mean(tempi), 1000 * np.std(tempi), REPEAT, NUMBER))

22.16 s ± 428 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [24]:
tf = CountVectorizer(max_features=1000000, lowercase=True, **params_tdm).fit(train_texts)
tf_train = tf.transform(train_texts)
ldax = LatentDirichletAllocation(max_iter=50, learning_method='online', random_state=0, **params_lda).fit(tf_train.toarray())
ldax_train = ldax.transform(tf_train.toarray())
tf_test = tf.transform(test_texts)
ldax_test = ldax.transform(tf_test.toarray())

params = {
    'max_depth':19,
    'min_samples_leaf':1 
}

def foo(params):
    ldac = DecisionTreeClassifier(random_state=0, **params).fit(ldax_train, train_cats)
    y_pred = ldac.predict(ldax_test)

REPEAT = 3
NUMBER = 1
tempi = timeit.repeat("foo(params)",setup="from __main__ import foo,params",repeat=REPEAT, number=NUMBER)
print("{:.2f} s ± {:.0f} ms per loop (mean ± std. dev. of {} runs, {} loop each)".format(
    np.mean(tempi), 1000 * np.std(tempi), REPEAT, NUMBER))

0.02 s ± 0 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


# Tempi senza LDA

In [25]:
tfc = Pipeline([
    ("classifier",DecisionTreeClassifier(random_state=0))
])

params_tdm = {
    'ngram_range': (1, 3),
    'min_df': 10,
    'max_df': 0.5
}

def foo(params_tdm):
    tf = CountVectorizer(max_features=1000000, lowercase=True, **params_tdm).fit(train_texts)
    tf_train = tf.transform(train_texts)
    tf_test = tf.transform(test_texts)
    
import timeit

REPEAT = 10
NUMBER = 1
tempi = timeit.repeat("foo(params_tdm)",setup="from __main__ import foo, params_tdm",repeat=REPEAT, number=NUMBER)
print("{:.2f} s ± {:.0f} ms per loop (mean ± std. dev. of {} runs, {} loop each)".format(
    np.mean(tempi), 1000 * np.std(tempi), REPEAT, NUMBER))

1.14 s ± 60 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [26]:
tf = CountVectorizer(max_features=1000000, lowercase=True, **params_tdm).fit(train_texts)
tf_train = tf.transform(train_texts)
tf_test = tf.transform(test_texts)

params = {
    'max_depth':29,
    'min_samples_leaf':1 
}

def foo(params):
    tfc = DecisionTreeClassifier(random_state=0, **params).fit(tf_train.toarray(), train_cats)
    y_pred = tfc.predict(tf_test.toarray())

REPEAT = 3
NUMBER = 1
tempi = timeit.repeat("foo(params)",setup="from __main__ import foo, params",repeat=REPEAT, number=NUMBER)
print("{:.2f} s ± {:.0f} ms per loop (mean ± std. dev. of {} runs, {} loop each)".format(
    np.mean(tempi), 1000 * np.std(tempi), REPEAT, NUMBER))

0.33 s ± 17 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
